In [1]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
%matplotlib inline
#matplotlib.rcParams['figure.figsize'] = (6, 4)
import seaborn as sns

# ignore pandas warnings
import warnings
warnings.simplefilter('ignore')

import time
start = time.time()

In [2]:
# load data
data = pd.read_csv('training_ultrasound.csv')

# remove agedays > 0 ( we just only focus pre-birth measurements)
data = data[data['AGEDAYS']<0]

# drop rows with missing data in any of the 5 main columns
ultrasound = ['HCIRCM', 'ABCIRCM', 'BPDCM', 'FEMURCM']
aux_measure = ['GAGEDAYS', 'SEXN', 'PARITY', 'GRAVIDA']
target = 'BWT_40'
data.dropna(subset=ultrasound+[target], inplace=True)

# correct faulty data
data.loc[data['STUDYID']==2, 'PARITY'] = data.loc[data['STUDYID']==2, 'PARITY'] + 1

## Model

In [3]:
# select basic vars
df = data[ultrasound + aux_measure + [target]]

In [4]:
df.isnull().sum()

HCIRCM        0
ABCIRCM       0
BPDCM         0
FEMURCM       0
GAGEDAYS      0
SEXN          0
PARITY      101
GRAVIDA     101
BWT_40        0
dtype: int64

In [5]:
# there is missing data for parity and gravida: this happens for first pregnancy --> fill with 1s
df.fillna(1, inplace=True)

# replace sex values to 0 and 1
df['SEXN'] = df['SEXN'].replace([1,2], [0,1])

### Feature engineering 

In [6]:
ultrasound

['HCIRCM', 'ABCIRCM', 'BPDCM', 'FEMURCM']

In [7]:
length_ratios = ['HCIRCM / ABCIRCM', 'HCIRCM / BPDCM', 'HCIRCM / FEMURCM',
                 'ABCIRCM / BPDCM', 'ABCIRCM / FEMURCM', 
                 'BPDCM / FEMURCM']
for ratio in length_ratios:
    df[ratio] = df[ratio.split(' ')[0]] / df[ratio.split(' ')[2]]

In [8]:
lenght_time = list()
for m in ultrasound:
    col_name = '%s / GAGEDAYS' % m
    lenght_time.append(col_name)
    df[col_name] = df[m] / df['GAGEDAYS']

lenght_time

['HCIRCM / GAGEDAYS',
 'ABCIRCM / GAGEDAYS',
 'BPDCM / GAGEDAYS',
 'FEMURCM / GAGEDAYS']

In [9]:
# no of past pregancies
df['past_gest'] = df['PARITY'] - df['GRAVIDA']

other_feat = ['past_gest'] 

In [10]:
# common models for sonographic fetal weight estimation use log of the weight
df['BWT_40'] = np.log(1 + df['BWT_40'])

In [11]:
print('Dataframe size: %s,%s' % (df.shape[0],df.shape[1]))

Dataframe size: 7928,20


In [12]:
# sklearn imports
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import mean_absolute_error

# xgboost
from xgboost import XGBRegressor

# custom 'library'
from aux_fun import plot_learning_curve, plot_validation_curve

### Split train/test data

In [13]:
pf = PolynomialFeatures(degree=4)
X_poly_ultrasounds = pf.fit_transform(df[ultrasound].values)
X_aux_measure = df[aux_measure].values
X_lenght_ratios = df[length_ratios].values
X_lenght_time = df[lenght_time].values
X_other_feat = df[other_feat].values

X = np.concatenate((X_poly_ultrasounds,X_aux_measure,X_lenght_ratios,X_lenght_time,X_other_feat),axis=1)

Y = df[target].values

In [14]:
poly_feat_names = [e.replace('x0','HCIRCM').replace('x1','ABCIRCM').replace('x2','BPDCM').replace('x3','FEMURCM')
              for e in pf.get_feature_names()]

In [15]:
all_feat_names = pd.Series(poly_feat_names + aux_measure + length_ratios + lenght_time + other_feat)

In [16]:
# train-test split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

### Define model pipeline

In [17]:
# worked quite well: (max_depth=20,learning_rate=0.03,n_estimators=1000,subsample=0.9)
xgb = XGBRegressor(max_depth=3,learning_rate=0.1,n_estimators=100,subsample=0.9)

#### CV strategy

In [18]:
kf = KFold(n_splits=5)

CV using all features:

In [19]:
scores = list()

for train_k, test_k in kf.split(x_train):
    xgb.fit(x_train[train_k],y_train[train_k])
    scores.append(mean_absolute_error(y_train[test_k], xgb.predict(x_train[test_k])))
    
base_cv_score = np.mean(scores)

print('Score: %0.4f +- %0.4f' % (np.mean(scores),2*np.std(scores)))

Score: 0.0537 +- 0.0027


Iterate through all features, calculating the disminution of error of leaving that one out, wrt to the base set of features:

In [20]:
score_after_deleting = list()
for ix, feat in all_feat_names.iteritems():
    score_i = cross_val_score(xgb, X=np.delete(x_train,[ix],axis=1), y=y_train, scoring='mean_absolute_error', cv=kf, n_jobs=-1)
    score_after_deleting.append(score_i)
    if (ix + 1) % 10 == 0:
        print('step %i done' % (ix + 1))
print('All done!')

step 10 done
step 20 done
step 30 done
step 40 done
step 50 done
step 60 done
step 70 done
step 80 done
All done!


Order features (features with negative error increments are more important):

In [21]:
feature_deltas = [np.abs(score.mean()) - base_cv_score for score in score_after_deleting]

sorted_feat_importances = \
pd.DataFrame({'names': all_feat_names, 'deltas': feature_deltas})[['names','deltas']].sort_values(by='deltas')

sorted_feat_importances.head(10)

,names,deltas
76,HCIRCM / FEMURCM,-0.000128
43,HCIRCM^2 BPDCM FEMURCM,-0.000102
47,HCIRCM ABCIRCM^2 FEMURCM,-0.000095
70,GAGEDAYS,-0.000091
53,HCIRCM BPDCM FEMURCM^2,-0.000084
84,past_gest,-0.000083
26,ABCIRCM^2 BPDCM,-0.000081
66,BPDCM^3 FEMURCM,-0.000079
32,BPDCM^2 FEMURCM,-0.000072
57,ABCIRCM^3 FEMURCM,-0.000071


In [22]:
sorted_feat_importances.to_excel('feature_importances.xlsx')
most_important_feat_indices = sorted_feat_importances.iloc[:30].index.tolist()

In [23]:
x_train_reduced = x_train[:,most_important_feat_indices]
x_test_reduced = x_test[:,most_important_feat_indices]

In [24]:
final_cv_score = cross_val_score(xgb, X=x_train_reduced, y=y_train, scoring='mean_absolute_error',
                                n_jobs=-1, cv=kf)

print('Score: %0.4f +- %0.4f' % (np.abs(final_cv_score.mean()),2*final_cv_score.std()))

Score: 0.0553 +- 0.0023


#### Fit whole train with best hyperparameter

In [25]:
xgb.fit(x_train_reduced,y_train)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=0.9)

In [26]:
print('Test mean abs error: ', mean_absolute_error(y_test, xgb.predict(x_test_reduced)))

Test mean abs error:  0.0529686669208


## Model assessment

In [27]:
# Compute true errors
w_true = np.exp(y_test) - 1
w_pred = np.exp(xgb.predict(x_test_reduced)) - 1
abs_error = np.absolute(w_true - w_pred)
mean_abs_error = abs_error.mean()
pct_error = abs_error / w_true

# true test set errors
print('Mean absolute error: %0.4f' % mean_abs_error)
print('Mean relative error: %0.4f' % pct_error.mean())

Mean absolute error: 0.2261
Mean relative error: 0.0695


#### Feature importances

In [28]:
time.time() - start

299.13517141342163